## Schrodinger Equation ##

We are going to solve the Schrodinger equation on a domain $\Omega_h$ with Dirichlet boundary contitions.

The equation to solve is:

$$
i \frac{\partial u}{\partial t} = \Delta u 
$$

Which we solve using,

$$
i (u - u_n) = dt *  \Delta (u + u_n)/2
$$

As time integrator, here $u = u_{n+1}$. Since the eigenvalues of $\Delta$ are proportional to $\Delta x^2$, $\lambda = -c/\Delta x^2$, $c > 0$, we must be carefull with the stability region of the method. We have, for each eigenfunction, 

$$
(i - \lambda dt/2)u = (i + \lambda dt /2 )u_n
$$

Thus, the amplification factor is: 

$$
|\frac{1 + i\lambda dt /2}{1 - i\lambda dt /2}| = 1
$$

and the system is unconditionaly stable.



We shall use its weak form,

$$
\int [2i v*u +  \nabla v \cdot \nabla u ] \; d\Omega = \int [2i*v*u_n - \nabla v \cdot \nabla u_n ] \; d\Omega
$$



In [76]:
import Pkg; Pkg.activate("gridap_makie")
using Gridap
using GridapMakie
using GLMakie GLMakie.activate()#, WGLMakie
using FileIO
#using Plots
#mkdir("models")
#mkdir("images")
using GridapGmsh
#using GridapGeosciences
using Gridap.Arrays
using Gridap.TensorValues
using Gridap.ReferenceFEs
using Gridap.Geometry
using Gridap.Fields
using Gridap.Algebra
using SparseArrays
using Gridap.FESpaces
using Gridap.CellData
using Gridap
using GridapMakie, #=GLMakie,=# WGLMakie
using FileIO
#using gmsh
using LinearAlgebra
using SuiteSparse
import Gmsh: gmsh

  Activating project at `~/Docencia/PDE/pde/2022/Numéricos/Tarea_7/gridap_makie`


LoadError: syntax: extra token "GLMakie" after end of expression

In [77]:
#include("../Tarea_5/mesh_generator.jl")
include("mesh_generator_Gmesh.jl")
grid_type = "double_slit"

Lx = 1.0
Ly = 1.0
Sx = 0.05
Sy = 0.4
SS = 0.05
#h = 0.01 #grosa not sure how it relates to h
#h = 0.1 #junta
h = 0.005 #
grid_name = "double_slit_grosa"; p = (grid_name, 1, 1, 0.05, 0.4, 0.05, h) #for test / rectangle (Length_x, Length_y, h) 
grid_name = "double_slit_junta"; p = (grid_name, 1, 1, 0.05, 0.3, 0.1, h)
grid_name = "double_slit"; p = (grid_name, 1, 1, 0.05, 0.25, 0.1, h)
grid_name = "double_slit_junta_dis"; p = (grid_name, Lx, Ly, Sx, Sy, SS, h) 
boundary_tags = ["ext","int"]
dirichlet_tags= ["ext","int"]
dirichlet_values = 0.0 # 0.0+im*0.0
model = make_model(grid_type, p)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 50%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 80%] Meshing curve 13 (Line)
Info    : [ 90%] Meshing curve 14 (Line)
Info    : [ 90%] Meshing curve 15 (Line)
Info    : [100%] Meshing curve 16 (Line)
Info    : Done meshing 1D (Wall 0.0167276s, CPU 0.001718s)
Info    : Meshing 2D...
Info    : Meshing surface 100 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 6.031s, CPU 5.31174s)
Info    : 93601 nodes 187218 elements
Info    : Writing 'models/double_slit_junta_dis.msh'...
Info    : Done writing 'models/double_slit

UnstructuredDiscreteModel()

In [78]:
T = 0.1            # final time for test
#T = 0.8
num_steps = 101   # number of time steps for 
dt = T / (num_steps-1) # time step size


# Intervals where data is saved
n_int = 5 # for 64
#mod = 60 # for 128
#mod = 120 # for 128 T=6
#mod = 240 # for 256 T=3
#mod = 480

#= DUMPING OF THE SOLUTION NEAR THE BOUDARIES SO THAT THERE IS NO REFLEXION =#
dumping = false
#dumping = true

if dumping
    save_file_1 = "Results/DoubleSlit/double_slit_junta_dumped"  #where to save things
else
    save_file_1 = "Results/DoubleSlit/double_slit_junta"
end

println("dt / dx^2 = $(dt / h^2)   dt / dx = $(dt / h)  dt^3 / dx^6 = $((dt / h^2)^3)")

dt / dx^2 = 40.0   dt / dx = 0.2  dt^3 / dx^6 = 64000.0


The first number indicates what it would be the CFL condition for the explicit Euler method. The second a guide about taking time and space comparable steps. The third is proportinal to the error, obtained by expanding $\frac{1 + i\lambda dt /2}{1 - i\lambda dt /2} - e^{i\lambda dt} = (1 + i\lambda dt /2)(1 + i\lambda dt /2 - \frac{\lambda^2 dt^2}{4})  - (1 + i\lambda dt - \frac{\lambda^2 dt^2}{2}) = O(\lambda dt)^3 $

In [79]:
order = 2
degree = order
Ωₕ = Triangulation(model)
dΩₕ = Measure(Ωₕ,degree)


Measure()

In [80]:
ue(x)=x[1]
Γ₁ = BoundaryTriangulation(model,tags=["ext", "int"])
fig, ax , plt  = plot(Γ₁,ue, colormap=:heat, linewidth=10)
ax.aspect = AxisAspect(2.2)
Colorbar(fig[1,2], plt)
fig

#plot(Γ₁)

In [81]:
#scatter(BoundaryTriangulation(model,tags=[4]))


In [82]:
reffe = ReferenceFE(lagrangian,Float64,order)

(Lagrangian(), (Float64, 2), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [83]:
V = TestFESpace(model,reffe,dirichlet_tags=dirichlet_tags,conformity=:H1,vector_type=Vector{ComplexF64})
U = TrialFESpace(V,0.0)

TrialFESpace()

In [84]:
#ls = LUSolver()
#solver = LinearFESolver(ls)

In [85]:
norm_L2(u) = sqrt(real(sum(∫(u'*u)*dΩₕ)))

norm_L2 (generic function with 1 method)

In [86]:
function u_0_particle(x,par)
    r0, xi, k, p = par
    r = sqrt((x[1]-xi[1])^2 + 0.2*(x[2]-xi[2])^2)
    kx = 2π*(k[1]*(x[1]-xi[1]) + k[2]*(x[2]-xi[2]))
    if r < r0
        return (r-r0)^p * 4^p *exp(im*kx)
    else 
        return 0.0 + 0.0*im
    end
end
        

par_particle = (0.2, [0.5;0.5], [-10;0], 2)

u_0_particle([0.55;0.55], par_particle)


-0.33745756319669346 + 8.578409431259935e-16im

In [87]:
u_0(x) = u_0_particle(x, par_particle)    


u_n = interpolate_everywhere(u_0,V)
#writevtk( Ωₕ,"Results/results_$(lpad(0,3,'0'))", cellfields=["uh_r"=>real(u_n),"uh_i"=>imag(u_n)])


SingleFieldFEFunction():
 num_cells: 185602
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 2118210642548503377

In [88]:
fig, axₕlt = plot(Ωₕ, real(u_n), colormap=:heat)
ax.aspect = AxisAspect(2.2)
Colorbar(fig[2,1], plt, vertical=false)
current_figure()
#fig

In [89]:
function ϵ(x)
    l = Lx*0.1
    lx_0 = l
    lx_1 = 2*Lx + Sx - l
    ly_0 = l
    ly_1 = Ly - l
    if (x[1] < lx_0)
        return 1 - (x[1]/lx_0)^2
    elseif (x[2] < ly_0)
        return 1 - (x[2]/ly_0)^2
    elseif (x[1] > lx_1)
        return 1 - ((x[1] - (lx_1+l))/l)^2
    elseif (x[2] > ly_1)
        return 1 - ((x[2] - (ly_1+l))/l)^2
    else
        return 0.0 + im*0.0
    end
end

if dumping 
    ϵₕ = interpolate_everywhere(ϵ,V)
else
    ϵₕ = interpolate_everywhere(0.0,V)
end

fig, axₕlt = plot(Ωₕ, real(ϵₕ), colormap=:heat)
ax.aspect = AxisAspect(2.2)
Colorbar(fig[2,1], plt, vertical=false)
current_figure()

To generate just once the matrix A we follow this example: https://github.com/gridap/Gridap.jl/blob/master/test/FESpacesTests/AssemblersTests.jl



In [90]:
assem = SparseMatrixAssembler(U,V)
dv = get_fe_basis(V)
du = get_trial_fe_basis(U)



a(u,v) = ∫(((1 + im*ϵₕ))*(∇(v) ⋅ ∇(u))*dt + im*2*v*u)dΩₕ

mat_contribs = a(du,dv)
data = collect_cell_matrix(U,V,mat_contribs)
A = assemble_matrix(assem,data)

369620×369620 SparseMatrixCSC{ComplexF64, Int64} with 4230644 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

In [91]:
t = 0
u_n = interpolate_everywhere(u_0,V)
writevtk(
          Ωₕ,save_file_1 * "_$(lpad(0,3,'0'))",
          cellfields=["uh_r"=>real(u_n),"uh_i"=>imag(u_n)])

    
for istep in 1:num_steps

    #println("\n+++ Solving in step $istep of $num_steps +++\n")
    t = t+dt
    b(v) = ∫(((1 + im*ϵₕ))*(∇(v) ⋅ ∇(-u_n))*dt + im*2*v*u_n)dΩₕ
    vec_contribs = b(dv)
    data = collect_cell_vector(V,vec_contribs)
    b = assemble_vector(assem,data)
    
    #op = AffineFEOperator(a,b,U,V)
    #uh = solve(solver,op)
    
    x = A \ b
    uh = FEFunction(U,x)
    
    u_n = uh
    
    if (istep-1) % n_int == 0 && istep > 1
        writevtk(
          Ωₕ, save_file_1 * "_$(lpad(istep,3,'0'))",
          cellfields=["uh_r"=>real(u_n),"uh_i"=>imag(u_n)])
        l2 = norm_L2(u_n)
        println("time = $t,      l2 = $l2")
    end
    
end

time = 0.006,      l2 = 0.08759486553220386
time = 0.011000000000000003,      l2 = 0.08759486553220365
time = 0.016000000000000007,      l2 = 0.08759486553220368
time = 0.02100000000000001,      l2 = 0.08759486553220587
time = 0.026000000000000016,      l2 = 0.08759486553220691
time = 0.03100000000000002,      l2 = 0.08759486553220688
time = 0.036000000000000025,      l2 = 0.08759486553220637
time = 0.04100000000000003,      l2 = 0.08759486553220701
time = 0.046000000000000034,      l2 = 0.08759486553220779
time = 0.05100000000000004,      l2 = 0.08759486553220705
time = 0.05600000000000004,      l2 = 0.08759486553220813
time = 0.06100000000000005,      l2 = 0.08759486553220687
time = 0.06600000000000004,      l2 = 0.0875948655322077
time = 0.07100000000000005,      l2 = 0.08759486553220815
time = 0.07600000000000005,      l2 = 0.08759486553220694
time = 0.08100000000000006,      l2 = 0.0875948655322086
time = 0.08600000000000006,      l2 = 0.0875948655322084
time = 0.09100000000000007